In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

## Import CSV as a DataFrame

In [2]:
##Bring in CSV Data
filepath = "Resources/DB-Vehicle-PTE.csv"
##Read CSV  
cars_df = pd.read_csv(filepath, encoding="ISO-8859-1")
##Use .head() to show first five rows
cars_df.head()

,Brand,Vehicle,Engine,Engine code,Torque (Nm/rpm),Power (hp - kW /rpm),Average fuel consumption (l/100 km),CO2 (g/km),Cx/S(m²)/SCx,Weight(3p/5p) kg,Displacement,Block material /cylinder head,Compression ratio,GearBox type,Tyre,Others,Traction type
0,Renault,Twingo,1.2L 60 hp,D7F,93/2500,58-43/5250,5.6,130,?,1000,1149,cast iron /aluminium,9.6,MT,165/65 R14,12V,2wd
1,Renault,Twingo,1.2L 75 hp,D4F,108/4250,75-55/5500,5.1,120,?,1025,1149,cast iron /aluminium,9.8,MT,165/65 R14,16V,2wd
2,Renault,Twingo,1.5L dCi 65 hp,K9KBP,160/1900,65-48/3750,4.3,113,?,1055,1461,cast iron /aluminium,17.9,MT,165/65 R14,8V+TC,2wd
3,Renault,Twingo,1.2L 75 hp AT,D4F,108/4250,75-55/5500,5.6,132,?,1030,1149,cast iron /aluminium,9.8,AT 5,165/65 R14,16V,2wd
4,Renault,Twingo,1.5L dCi 85 hp,K9KHP,200/1750,84-62/3750,4,104,?,1055,1461,cast iron /aluminium,17.9,MT,165/65 R14,8V+TC+CAC,2wd


## Create new dataframe with columns to keep

In [3]:
#Drop unwanted columns
cleaned_cars_df = cars_df[['Brand', 'Vehicle', 'Engine', 'Power (hp - kW /rpm)', 'Average fuel consumption (l/100 km)', 'CO2 (g/km)', 'Weight(3p/5p) kg', 'Block material /cylinder head', 'Traction type']]
#Showing first 5 rows
cleaned_cars_df.head()

,Brand,Vehicle,Engine,Power (hp - kW /rpm),Average fuel consumption (l/100 km),CO2 (g/km),Weight(3p/5p) kg,Block material /cylinder head,Traction type
0,Renault,Twingo,1.2L 60 hp,58-43/5250,5.6,130,1000,cast iron /aluminium,2wd
1,Renault,Twingo,1.2L 75 hp,75-55/5500,5.1,120,1025,cast iron /aluminium,2wd
2,Renault,Twingo,1.5L dCi 65 hp,65-48/3750,4.3,113,1055,cast iron /aluminium,2wd
3,Renault,Twingo,1.2L 75 hp AT,75-55/5500,5.6,132,1030,cast iron /aluminium,2wd
4,Renault,Twingo,1.5L dCi 85 hp,84-62/3750,4,104,1055,cast iron /aluminium,2wd


## Reset index to make primary key

In [4]:
##Using .reset_index to make 'id' our primary key
cleaned_cars_df = cleaned_cars_df.reset_index(drop=False)

## Rename columns

In [5]:
##Use .rename to give columns new headers
cleaned_cars_df = cleaned_cars_df.rename(columns={'index': 'id',
                                                  'Brand': 'brand',
                                                  'Vehicle': 'vehicle', 
                                                  'Engine': 'engine',
                                                  'Power (hp - kW /rpm)': 'power',
                                                 'Average fuel consumption (l/100 km)': 'average_fuel_consumption',
                                                 'CO2 (g/km)': 'co2',
                                                 'Weight(3p/5p) kg': 'weight', 
                                                  'Block material /cylinder head': 'material', 
                                                  'Traction type': 'traction_type'})
#Showing first five rows
cleaned_cars_df.head()

,id,brand,vehicle,engine,power,average_fuel_consumption,co2,weight,material,traction_type
0,0,Renault,Twingo,1.2L 60 hp,58-43/5250,5.6,130,1000,cast iron /aluminium,2wd
1,1,Renault,Twingo,1.2L 75 hp,75-55/5500,5.1,120,1025,cast iron /aluminium,2wd
2,2,Renault,Twingo,1.5L dCi 65 hp,65-48/3750,4.3,113,1055,cast iron /aluminium,2wd
3,3,Renault,Twingo,1.2L 75 hp AT,75-55/5500,5.6,132,1030,cast iron /aluminium,2wd
4,4,Renault,Twingo,1.5L dCi 85 hp,84-62/3750,4,104,1055,cast iron /aluminium,2wd


In [6]:

cleaned_cars_df.columns

Index(['id', 'brand', 'vehicle', 'engine', 'power', 'average_fuel_consumption',
       'co2', 'weight', 'material', 'traction_type'],
      dtype='object')

## Remove rows with missing values

In [7]:
cols = ['id', 'brand', 'vehicle', 'engine', 'power', 'average_fuel_consumption', 'co2', 'weight', 'material', 'traction_type']
##Performed a loop to make sure all columns with ? is dropped
for col in cols:
    cleaned_cars_df = cleaned_cars_df[cleaned_cars_df[col] != '?']
cleaned_cars_df

,id,brand,vehicle,engine,power,average_fuel_consumption,co2,weight,material,traction_type
0,0,Renault,Twingo,1.2L 60 hp,58-43/5250,5.6,130,1000,cast iron /aluminium,2wd
1,1,Renault,Twingo,1.2L 75 hp,75-55/5500,5.1,120,1025,cast iron /aluminium,2wd
2,2,Renault,Twingo,1.5L dCi 65 hp,65-48/3750,4.3,113,1055,cast iron /aluminium,2wd
3,3,Renault,Twingo,1.2L 75 hp AT,75-55/5500,5.6,132,1030,cast iron /aluminium,2wd
4,4,Renault,Twingo,1.5L dCi 85 hp,84-62/3750,4,104,1055,cast iron /aluminium,2wd
...,...,...,...,...,...,...,...,...,...,...
1253,1253,Audi,Q5,2.0L TDI 170 hp DPF quattro S-tronic 7,170-125/4200,6.8,179,1845,cast iron /aluminium,4wd
1254,1254,Audi,Q5,3.0L V6 TDI 239 hp DPF quattro S-tronic 7,239-176/4400,7.5,199,1940,cast iron /aluminium,4wd
1255,1255,Audi,Q5,2.0L TFSI 211 hp quattro,211-155/6000,8.3,192,1770,cast iron /aluminium,4wd
1256,1256,Audi,Q5,2.0L TFSI 211 hp quattro S-tronic 7,211-155/6000,8.5,197,1815,cast iron /aluminium,4wd


## Create connection

In [8]:
##Creating a connection with postgres using password file
connection_string = "postgres:" + password + "@localhost:5432/cars_db"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
##Confirm our table
engine.table_names()

C:\Users\apfle\AppData\Local\Temp\ipykernel_27316\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['cars']

## Load into Postgresql

In [11]:
##We are loading our Data into Postgres using .to_sql with name of Table
cleaned_cars_df.to_sql(name='car', con=engine, if_exists='append', index=False)

134